# 작사하는 인공지능 모델 만들기

**Context**
1. 시퀀스란?
2. I 다음엔 am이 오면 반 이상은 맞더라
3. 데이터 다듬기
4. 인공지능 학습시키기
5. 모델 평가하기

## 시퀀스란?

시퀀스, 다들 익숙하신 단어일 것 같아요. 근데 이게 어디에서 쓰이는 단어냐에 따라 그 의미가 조금씩 차이가 있을 것 같습니다. 그럼 인공지능에서의 시퀀스는 무엇일까요? 아래 글을 읽오보고 옵시다!

- [파이썬 프로그래밍 입문서(가제) 5.2 시퀀스](https://python.bakyeono.net/chapter-5-2.html)

시퀀스 데이터가 곧 각 요소들의 연관성을 의미하는 것은 아니지만 인공지능이 예측을 하려면 *어느정도는 연관성이 있어줘야 합니다.* 예를 들어 `[17.09.01, 18.09.01, 19.09.01, ?]` 이런 시퀀스가 있을 때, 우리는 ?의 값이 `20.09.01`이라고 예측할 수 있겠죠. 

문장을 구성하는 각 단어들은 문법이라는 규칙에 따라 배열되어 있습니다. 아직 우리도 모든 문법을 다 숙지하지 못하고 있는데, 그런 의미에서 문장이라는 시퀀스 데이터는 꽤나 어렵습니다. 그래서 문장 시퀀스에서는 보통 **통계에 기반한 방법**을 사용합니다. 통상적으로, I 다음에는 am이 오는 경우가 많다! 이런 식인거죠!


## I 다음 am이 오면 반 이상은 맞더라

`나는 밥을 [  ]`, `알바생이 커피를 [   ]`

이 두 문장에서 `[ ]`에는 각각 무엇이 올까요? 네, 여러분이 짐작하는대로 먹는다, 만든다 등이 될 것 같습니다.

인공지능이 글을 이해하게 하는 방식도 위와 같습니다. 어떤 문법적인 원리가 아닌 <u>수많은 글을 읽게 함으로써</u> 그 다음에 오는 단어를 알 수 있도록 하는 것이죠. 그런 이유로 많은 데이터가 곧 좋은 결과를 만들어냅니다. 단어를 적재적소에 활용하는 능력을 학습시키는 것이죠!

이 방식을 가장 잘 처리하는 인공지능 모델은 무엇일까요? 바로 **순환신경망(RNN)**입니다. RNN에 대한 자세한 내용은 아래 글에서 확인해봅시다.

- [Recurrent Neural Network (RNN) 이해하기](https://dreamgonfly.github.io/blog/understanding-rnn/)

우리는 문장을 이런식으로 만드려고 합니다. 

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-12-RNN2.max-800x600.png)

위는 순환신경망의 작동 방법을 가장 단순하게 표현한 그림입니다. 

`<start>`를 문장의 맨 앞에 추가해주어 문장의 시작을 알리고 하나씩 생성하는 단어를 봅니다. 그렇게 순차적으로 `나는 밥을 먹었다`까지 생성하고 나면, 다 만들었다는 사인으로 `<end>`라는 토큰을 생성합니다. `<start>`가 문장의 시작에 더해진 입력 데이터와 `<end>`가 문장 끝에 더해진 출력 데이터가 필요하며, 이는 문장 데이터만 있으면 만들어낼 수 있습니다.

이 내용을 코드로 표기하면 아래와 같습니다.

In [1]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


언어 모델 (Language Model)
나는, 밥을, 먹었다 를 순차적으로 생성할 때, 밥을 다음이 먹었다 인 것은 쉽게 알 수 있습니다. 하지만 나는 다음이 밥을 인 것은 조금 억지처럼 느껴질 수 있습니다. 실제로 동작하는 방식도, 밥을 을 만드는 것은 순전히 운입니다. 우리가 의도한다고 나오는 것이 아니죠.

이걸 좀더 확률적으로 표현해 보겠습니다. '나는 밥을' 다음에 '먹었다' 가 나올 확률을 $p(먹었다 | 나는, 밥을)$ 이라고 합시다. 그렇다면 이 확률은 '나는' 뒤에 '밥이' 가 나올 확률인 $p(밥을|나는)$보다는 높게 나올 것입니다. 아마 $p(먹었다 | 나는, 밥을, 맛있게)$의 확률값은 더 높아지겠죠?


어떤 문구 뒤에 다음 단어가 나올 확률이 높다는 것은 그 다음 단어가 나오는 것이 보다 자연스럽다는 뜻이 됩니다. 그렇다면 '나는' 뒤에 '밥을'이 나오는 것이 자연스럽지 않다는 뜻일까요? 그것은 아닙니다. '나는' 뒤에 올 수 있는 자연스러운 단어의 경우의 수가 워낙 많다 보니 불확실성이 높을 뿐입니다.

n-1개의 단어 시퀀스 $w_1, \cdots, w_{n-1}$ 가 주어졌을 때, n번째 단어 $w_n$ 으로 무엇이 올지를 예측하는 확률 모델을 **언어 모델(Language Model)**이라고 부릅니다. 파라미터 $θ$ 로 모델링하는 언어 모델을 다음과 같이 표현할 수 있습니다.

$$P(w_n | w_1, …, w_{n-1};\theta )$$

잠깐 스크롤을 올려 RNN의 개념도를 잠깐 다시 보시면, 정확히 $w_1, \cdots, w_{n-1}$
가 주어졌을 때, n번째 단어 $w_n$ 으로 무엇이 올지 예측하는 구조를 가지고 있음을 알아챌 수 있으실 겁니다. 이런 언어 모델을 어떻게 학습시킬 수 있을까요? 간단합니다. 어떤 텍스트도 언어 모델의 학습 데이터가 될 수 있습니다. n-1번째까지의 단어 시퀀스가 x_train이 되고 n번째 단어가 y_train이 되는 데이터셋은 무궁무진하게 만들 수 있으니까요. 이렇게 학습된 언어 모델을 학습 모드가 아닌 테스트 모드로 가동하면 어떤 일이 벌어질까요? 네, 이 모델은 일정한 단어 시퀀스가 주어진다면 다음 단어, 그 다음 단어를 계속해서 예측해 낼 것입니다. 이게 바로 텍스트 생성이고 작문 아니겠습니까? **잘 학습된 언어 모델은 훌륭한 문장 생성기**로 동작하게 됩니다.

이전 스텝에서 소개했던 GPT-2 같은 문장 생성기도 언어 모델의 한 종류에 불과합니다. 딥러닝 모델의 구조나 파라미터 사이즈, 학습데이터의 양 등이 특별할 뿐, 기본적인 원리는 오늘 우리가 만들게 될 언어 모델과 전혀 다를게 없습니다.

## 데이터 다듬기

그럼 이제 실습을 해봅시다. 셰익스피어의 대사를 학습해서 스스로 연극 대사 문장을 생성해내는 언어 모델 인공지능을 만들어볼 것입니다. 텐서플로우가 기본적으로 제공하고 있습니다.

데이터를 아래 명령어로 다운받고 `./data` 폴더로 옮겨주고 데이터를 불러와봅시다.

```bash
$ wget https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
```

In [5]:
import re
import numpy as np
import tensorflow as tf
import os

# 파일을 읽기모드로 열어봅시다.
file_path = os.path.dirname(os.path.abspath('__file__')) + r'/data/shakespeare.txt'
with open(file_path, 'r') as f:
    raw_corpus = f.read().splitlines() # 텍스트를 라인 단위로 끊어서 리스트 형태로 읽어옵니다.

print(raw_corpus[:9]) # 10개만 읽어봅시다.

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


첫번째 시민이 'Before..' 문장을 말하고, 모든 사람이 'Speak, Spaek,,' 이런 식으로 말하고 있네요. 완벽한 연극 대본입니다.

화자도 좋은 정보지만, 우리는 **문장(대사)**만을 원하므로 화자 이름이나 공백뿐인 정보는 필요하지 않습니다. 이런 부분을 처리해줍시다.

In [8]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: # 빈문장이면 넘어감
        continue
    if sentence[-1] == ":": # 화자를 말하는 것이면 넘어감
        continue
    if idx > 9:
        break
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


우리가 원하는 문장만 성공적으로 출력합니다.

> 토큰화 (Tokenize)

텍스트 분류 모델에서 많이 보신 것처럼 텍스트 생성 모델에도 단어 사전을 만듭니다. 이를 위해 문장을 일정한 기준으로 자르는데, 이 과정을 **토큰화(Tokenize)**라고 합니다.

가장 심플한 방법은 띄어쓰기를 기준으로 나누는 방법입니다. 하지만 몇가지 문제 케이스가 있습니다.

1. 특수문자가 들어갈 수 있다.
2. 대소문자를 다르게 분류한다.
3. `ten-year-old`와 같은 경우 `ten-year-old`를 한 단어로 인식한다.

(1)을 위해 문장 부호의 양쪽에 공백을 추가하고 (2)를 막기위해 모든 문자를 소문자로 변환해줍시다. (3)을 막기 위해 특수문자는 모두 제거하도록 합시다.

이런 전처리를 위해 정규표현식(Regex)를 이용해 필터링을 해봅시다.

In [9]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


짜잔, 지저분한 문장을 넣어도 예쁘게 변환해주는 정제 함수가 완성되었습니다! 보너스로 이전 스텝에서 배운 <start> <end> 도 추가했습니다.

그러면 우리가 구축해야 할 데이터셋은 어떤 모양이 될까요?

이전 스텝에서 봤던 예를 떠올려 봅시다.

```html
언어 모델의 입력 문장 :  <START> 나는 밥을 먹었다
언어 모델의 출력 문장 : 나는 밥을 먹었다 <END>    
```
    
자연어처리 분야에서 모델의 입력이 되는 문장을 **소스 문장(Source Sentence)**, 정답 역할을 하게 될 모델의 출력 문장을 **타겟 문장(Target Sentence)**라고 관례적으로 부릅니다. 각각 X_train, y_train 에 해당한다고 할 수 있겠죠?

그렇다면 우리는 위에서 만든 정제 함수를 통해 만든 데이터셋에서 토큰화를 진행한 후 끝 단어 를 없애면 소스 문장, 첫 단어 를 없애면 타겟 문장이 되겠죠? 이 정제 함수를 활용해서 아래와 같이 정제 데이터를 구축합니다!

In [10]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

이제 데이터는 완벽하게 준비가 된 것 같네요!

자, 새로운 언어를 배우는 상상을 해봅시다. 영어를 전혀 모르던 그 때로 돌아가서, 다시 영어를 배우려면 어떻게 해야 할까요? 영한사전 을 허리춤에 끼고 문장 속 단어를 하나하나 찾아가며 한국어 해석 을 적겠죠? 이 아이디어는 인공지능에게도 똑같이 적용됩니다. 배우고자 하는 언어 를 모국어로 표현 을 해야 공부를 할 수 있어요.

인공지능의 모국어라면 단연 숫자겠죠. 우리는 가르칠 언어(데이터)를 숫자로 변환해서 인공지능에게 줄 겁니다. 이에 필요한 것은 사전! 굳이 명명하자면… 데숫사전…?

텐서플로우는 자연어 처리를 위한 여러 가지 모듈을 제공하는데, 우리도 그 모듈을 십분 활용할 겁니다! 아래에서 활용하게 될 `tf.keras.preprocessing.text.Tokenizer` 패키지는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 한 방에 해줍니다. 이 과정을 **벡터화(vectorize)** 라 하며, 숫자로 변환된 데이터를 **텐서(tensor)** 라고 칭합니다. 우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리되는 것입니다.

텐서(Tensor)는 굉장히 어려운 물리학 및 수학 개념입니다. 텐서에 대해 모두 이해할 필요는 없으나 아래 웹페이지가 설명하는 간단한 개념 정도는 알고있으면 좋을 것 같습니다.

- [Tensor란 무엇인가?](https://rekt77.tistory.com/102)


In [11]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 7000, # 전체 단어의 개수
        filters = '', # 별도로 전처리 로직을 추가할 수 있습니다. 
        oov_token = '<unk>' # out of vocabulary. 사전에 없던 단어는 어떤 토큰으로 대체할지를 말합니다.
    )
    tokenizer.fit_on_texts(corpus) # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동 구축합니다.
    
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus) # tokenizer는 구축한 사전으로부터 corpus를 해석해 tensor로 반환
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드를 제공
    # maxlen의 디폴트 값은 none. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f1969121e10>


생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해 봅시다.


In [13]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


텐서 데이터는 모두 정수로 이루어져 있습니다. 이 숫자는 다름이 아니라, tokenizer에 구축된 단어 사전의 인덱스입니다. 단어 사전의 내용도 한번 확인해봅시다.

In [14]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10:
        break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


어떻습니까? 2번 인덱스가 바로 였습니다. 왜 모든 행이 2로 시작하는지 이해할 수 있겠습니다.

이제 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게 하겠습니다. 이 과정도텐서플로우가 제공하는 모듈을 사용할 것이니, 어떻게 사용하는지만 눈여겨 봐둡시다.

텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워넣은 것입니다. 사전에는 없지만 0은 바로 패딩 문자 가 될 것입니다.



In [15]:
src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내어 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 큽니다.
tgt_input = tensor[:, 1:] # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


corpus 내의 첫번째 문장에 대해 생성된 소스와 타겟 문장을 확인해 보았습니다. 예상대로 소스는 2()에서 시작해서 3()으로 끝난 후 0()로 채워져 있습니다. 하지만 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한칸 시프트한 형태를 가지고 있습니다.

#### `tf.data.Dataset` 객체

마지막으로 우리는 데이터셋 객체를 생성할 것입니다. 그동안 우리는 `model.fit(xtrain, ytrain, …)` 형태로 Numpy Array 데이터셋을 생성하여 model에 제공하는 형태의 학습을 많이 진행해 왔습니다. 그러나 텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 `tf.data.Dataset`객체를 생성하는 방법을 흔히 사용합니다. `tf.data.Dataset`객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의기능을 제공하므로 꼭 사용법을 알아 두시기를 권합니다. 

우리는 이미 데이터셋을 텐서 형태로 생성해 두었으므로, `tf.data.Dataset.from_tensor_slices()` 메소드를 이용해 `tf.data.Dataset`객체를 생성할 것입니다.

In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 # tokenizer가 구축한 단어사전 내 7000개와 여기 포함되어있지 않은 0 : <pad>를 포함하여 70001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

이번 스텝에서 데이터셋을 생성하기 위해 거쳐 온 과정을 잘 기억해 두시길 바랍니다.

- 정규표현식을 이용한 corpus 생성
- `tf.keras.preprocessing.text.Tokenizer`를 이용해 corpus를 텐서로 변환
- `tf.data.Dataset.from_tensor_slices()`를 이용해 corpus 텐서를 `tf.data.Dataset` 객체로 변환

`dataset`을 얻음으로써 데이터 다듬기 과정은 끝났습니다. `tf.data.Dataset`에서 제공하는 `shuffle()`, `batch()` 등 다양한 데이터셋 관련 기능을 손쉽게 사용할 수 있게 되었습니다. 

이 모든 일련의 과정을 텐서플로우에서의 **데이터 전처리**라 칭합니다.

## 인공지능 학습시키기

우리가 만들 모델의 구조도는 아래와 같습니다.

![IMG](https://aiffelstaticprd.blob.core.windows.net/media/images/E-12-4.max-800x600.png)

우리가 만들 모델은 `tf.keras.Model`을 Subclassing 하는 방식으로 만드려고 합니다. 위 그림에서 설명한 것처럼 우리가 만들 모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있습니다.



In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words+1, embedding_size, hidden_size)

텍스트 분류 모델을 다루어 보셨다면 Embedding 레이어의 역할에 대해서는 낯설지 않을 것입니다. 우리 입력 텐서에는 단어 사전의 인덱스가 들어 있습니다. Embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 줍니다. 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용됩니다.

위 코드에서 embedding_size 는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기입니다. 만약 그 크기가 2라면 예를 들어

- 차갑다: [0.0, 1.0]
- 뜨겁다: [1.0, 0.0]
- 미지근하다: [0.5, 0.5]

정도의 구분이 가능하겠군요. 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기할 수 있습니다. 이번 실습에서는 256이 적당해 보이네요.

LSTM 레이어의 `hidden state` 의 차원수인 `hidden_size` 도 같은 맥락입니다. hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가? 로 이해해도 크게 엇나가지 않습니다. 그 일꾼들은 모두 같은 데이터를 보고 각자의 생각을 가지는데, 역시 충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 갈 뿐 입니다. 이번 실습에는 1024가 적당해보이는군요.

우리의 model은 아직 제대로 build되지 않았습니다. `model.compile()`을 호출한 적도 없고, 아직 model의 입력 텐서가 무엇인지 제대로 지정해 주지도 않았기 때문입니다.
그런 경우 아래와 같이 model에 데이터를 아주 조금 태워 보는 것도 방법입니다. model의 input shape가 결정되면서 `model.build()`가 자동으로 호출됩니다.

In [20]:
for src_sample, tgt_sample in dataset.take(1):
    break
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-7.61025658e-05, -4.82523756e-04,  2.77600542e-04, ...,
         -1.36298622e-04, -3.55820055e-04,  2.49736360e-04],
        [ 2.89597345e-04, -6.41959021e-04,  3.03645036e-04, ...,
         -9.02097236e-06, -5.00052120e-04,  4.21056000e-04],
        [ 6.03727880e-04, -7.69524195e-04,  3.96174262e-04, ...,
          7.95719898e-05, -4.20393393e-04,  5.18605288e-04],
        ...,
        [-3.53383669e-03,  3.38338851e-03,  2.47288239e-03, ...,
          6.22861611e-04, -1.75298541e-03, -2.51874141e-03],
        [-3.61233926e-03,  3.65450862e-03,  2.88525899e-03, ...,
          6.75261079e-04, -1.86508428e-03, -2.79006688e-03],
        [-3.67266056e-03,  3.86501616e-03,  3.27356509e-03, ...,
          7.31412496e-04, -1.98015035e-03, -3.02895112e-03]],

       [[-7.61025658e-05, -4.82523756e-04,  2.77600542e-04, ...,
         -1.36298622e-04, -3.55820055e-04,  2.49736360e-04],
        [-7.05882412e-05, -6.20090752e-04,  2.

모델의 최종 출력 텐서 shape를 유심히 보면 `shape=(256, 20, 7001)`임을 알 수 있습니다. 7001은 Dense 레이어의 출력 차원수입니다. 7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 하기 때문입니다.
256은 이전 스텝에서 지정한 배치 사이즈입니다. `dataset.take(1)`를 통해서 1개의 배치, 즉 256개의 문장 데이터를 가져온 것입니다.

그렇다면 20은 무엇을 의미할까요? 비밀은 바로 `tf.keras.layers.LSTM(hidden_size, return_sequences=True)`로 호출한 LSTM 레이어에서 `return_sequences=True`이라고 지정한 부분에 있습니다. 즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미입니다. 만약 `return_sequences=False`였다면 LSTM 레이어는 1개의 벡터만 출력했을 것입니다. 그런데 문제는, 우리의 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모른다는 점입니다. 모델을 만들면서 알려준 적도 없습니다. 그럼 20은 언제 알게된 것일까요? 네, 그렇습니다. 데이터를 입력받으면서 비로소 알게 된 것입니다. 우리 데이터셋의 max_len이 20으로 맞춰져 있었던 것입니다.

In [21]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


이제 드디어 `model.summary()`를 호출할 수 있게 되었습니다. 그런데 호출해 보니 그동안 많이 보았던 것과는 다른 점이 있습니다. 우리가 궁금했던 Output Shape를 정확하게 알려주지 않습니다. 바로 위에서 설명한 이유 때문입니다. 우리의 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없는 것입니다.

하지만 모델의 파라미터 사이즈는 측정됩니다. 대략 22million 정도 되는군요. 

이제 모델이 학습할 준비가 완료되었습니다. 아래 코드를 실행해 모델을 학습시켜보세요!

In [22]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction = 'none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 14s 147ms/step - loss: 3.5016
Epoch 2/30
93/93 [==============================] - 13s 145ms/step - loss: 2.8117
Epoch 3/30
93/93 [==============================] - 13s 141ms/step - loss: 2.7260
Epoch 4/30
93/93 [==============================] - 13s 138ms/step - loss: 2.6419
Epoch 5/30
93/93 [==============================] - 13s 139ms/step - loss: 2.5716
Epoch 6/30
93/93 [==============================] - 13s 139ms/step - loss: 2.5209
Epoch 7/30
93/93 [==============================] - 13s 142ms/step - loss: 2.4673
Epoch 8/30
93/93 [==============================] - 15s 165ms/step - loss: 2.4143
Epoch 9/30
93/93 [==============================] - 16s 172ms/step - loss: 2.3693
Epoch 10/30
93/93 [==============================] - 15s 163ms/step - loss: 2.3285
Epoch 11/30
93/93 [==============================] - 16s 171ms/step - loss: 2.2889
Epoch 12/30
93/93 [==============================] - 16s 167ms/step - loss: 2.2495
Epoch 13/30
9

Loss는 모델이 오답을 만들고 있는 정도라고 생각하셔도 좋습니다(그렇다고 Loss가 1일 때 99%를 맞추고 있다는 의미는 아닙니다). 오답률이 감소하고 있으니 학습이 잘 진행되고 있다 고 해석할 수 있죠!

학습이 완료되었다면 이제 모델을 평가해 봅시다.

## 모델 평가하기

모델이 작문을 잘하는지 컴퓨터 알고리즘이 평가하는 것은 무리가 있습니다. 작문 모델을 평가하는 가장 확실한 방법은 작문을 시켜보고 직접 평가하는 것입니다. 아래 `generate_text` 함수를 만들어 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 해봅시다.

In [29]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력합니다.
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.
        
        # 우리 모델이 새롭계 예측한 단어를 입력 문장의 뒤에 붙여줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면 while 루프를 돌면서 다음 단어를 예측해야합니다.
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break
    
    generated = ""
    
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

텍스트를 생성하는 함수 안을 들여다보면 while문이 하나 자리잡고 있는 것을 볼 수 있습니다. 왜 그럴까요?

학습 단계에서 우리는 이런 while 문이 필요없었습니다. 소스 문장과 타겟 문장이 있었고, 우리는 소스 문장을 모델에 입력해서 나온 결과를 타겟 문장과 직접 비교하면 그만이었습니다.
그러나 텍스트를 실제로 생성해야 하는 시점에서, 우리에게는 2가지가 없습니다. 하나는 타겟 문장입니다. 또하나는 무엇이냐 하면, 소스 문장입니다. 생각해 보면 우리는 텍스트 생성 태스크를 위해 테스트 데이터셋을 따로 생성한 적이 없습니다.

`generate_text()` 함수에서 `init_sentence`를 인자로 받고는 있습니다. 이렇게 받은 인자를 일단 텐서로 만들고 있습니다. 디폴트로는 단어 하나만 받는군요.

- while의 첫번째 루프에서 test_tensor에 `<START>` 하나만 들어갔다고 합시다. 우리의 모델이 출력으로 7001개의 단어 중 A를 골랐다고 합시다.
- while의 두번째 루프에서 test_tensor에는 `<START>` A가 들어갑니다. 그래서 우리의 모델이 그다음 B를 골랐다고 합시다.
- while의 세번째 루프에서 test_tensor에는 `<START>` A B가 들어갑니다. 그래서….. (이하 후략)


그럼 실제로 위 문장 생성 함수를 실행해 볼까요?

In [30]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he is not fourteen , and i am not <end> '

재밌는 문장을 생성했습니다!! 다른 문장도 한번 해볼까요?

In [38]:
generate_text(model, tokenizer, init_sentence="<start> if I")

'<start> if i have heard , i ll have you talked on the city <end> '

문법적으로 좀 어색하지만, 어쨌든 문장같이 나오네요!

이것으로 실습을 마치도록 하겠습니다. :)

In [39]:
from IPython.core.display import display, HTML
display(HTML("<style> .container{width:90% !important;}</style>"))